# 🚀 Entrenamiento Deformable DETR en Google Colab - V1 (Optimizado)

Este notebook entrena un modelo Deformable DETR con MMDetection en Google Colab para detección de vida silvestre aérea.

## 🔥 **MEJORAS V1:**
- ✅ **Guardado automático en Drive** cada X épocas
- ✅ **Recuperación de entrenamiento** interrumpido
- ✅ **Configuración optimizada** para velocidad
- ✅ **Monitoreo en tiempo real** del progreso
- ✅ **Backup automático** de checkpoints

## 📋 Características
- **Modelo**: Deformable DETR con backbone Swin-T o ResNet-50
- **Framework**: MMDetection (OpenMMLab)
- **Detección automática de GPU**
- **Configuración automática de datasets**
- **Visualización de resultados**
- **Exportación de modelos**

## 🎯 Clases de Animales
- Buffalo
- Elephant  
- Kob
- Alcelaphinae
- Warthog
- Waterbuck

## ⚙️ Configuración por Defecto
- **Backbone**: Swin-T (recomendado) o ResNet-50
- **Tamaño de imagen**: 896x896
- **Épocas**: 50
- **Batch size**: 4 (train), 2 (val/test)
- **Optimizador**: AdamW con weight decay

## 🔬 Ventajas de Deformable DETR
- **End-to-end**: Sin necesidad de NMS post-procesamiento
- **Eficiente**: Menos parámetros que R-CNN basados en regiones
- **Preciso**: Mejor rendimiento en objetos pequeños
- **Flexible**: Arquitectura transformer moderna


## 🔧 Instalación de Dependencias


In [ ]:
# Instalar dependencias principales
%pip install -q openmim
!mim install -q mmcv-full
!mim install -q mmdet
%pip install -q pyyaml opencv-python pillow tqdm matplotlib seaborn pandas

# Verificar instalación
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")


## 📦 Importar Librerías


In [ ]:
# ============================================================
# VARIABLES GLOBALES PARA BACKUP AUTOMÁTICO
# ============================================================

# Variables globales para el sistema de backup
backup_thread_running = False
backup_thread = None

print("✅ Variables globales de backup inicializadas")


In [ ]:
import os
import sys
import yaml
import torch
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import shutil
import json
from tqdm import tqdm

# MMDetection imports
from mmengine.config import Config
from mmengine.runner import Runner
import mmdet
from mmdet.apis import init_detector, inference_detector
from mmdet.utils import register_all_modules

# Google Colab
from google.colab import files, drive

# Configurar matplotlib
plt.style.use('default')
sns.set_palette("husl")

print(f"MMDetection version: {mmdet.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


## 📁 Configuración de Datos


In [ ]:
# ============================================================
# CONFIGURACIÓN DE RUTAS PRINCIPALES (Unificado con HerdNet)
# ============================================================

# Definir la ruta base principal (ajustar según el entorno: Drive o local)
BASE_DIR = Path("/content/drive/MyDrive/aerial-wildlife-count")

# ============================================================
# RUTAS DE IMÁGENES Y ANOTACIONES (COCO JSON) - Igual que HerdNet
# ============================================================

# Rutas de los archivos de anotaciones en formato COCO
TRAIN_ANN_FILE = BASE_DIR / "data" / "coco" / "train" / "train_annotations.json"
VAL_ANN_FILE = BASE_DIR / "data" / "coco" / "val" / "val_annotations.json"
TEST_ANN_FILE = BASE_DIR / "data" / "coco" / "test" / "test_annotations.json"

# Rutas de las carpetas de imágenes correspondientes a cada conjunto
TRAIN_IMG_DIR = BASE_DIR / "data" / "images" / "train"
VAL_IMG_DIR = BASE_DIR / "data" / "images" / "val"
TEST_IMG_DIR = BASE_DIR / "data" / "images" / "test"

# ============================================================
# RUTAS ALTERNATIVAS (Fallback para compatibilidad)
# ============================================================

# Rutas alternativas si no existe la estructura estándar
TRAIN_ANN_FILE_ALT = BASE_DIR / "data" / "groundtruth" / "json" / "big_size" / "train_big_size_A_B_E_K_WH_WB.json"
VAL_ANN_FILE_ALT = BASE_DIR / "data" / "groundtruth" / "json" / "big_size" / "val_big_size_A_B_E_K_WH_WB.json"
TEST_ANN_FILE_ALT = BASE_DIR / "data" / "groundtruth" / "json" / "big_size" / "test_big_size_A_B_E_K_WH_WB.json"

# Rutas alternativas para imágenes
TRAIN_IMG_DIR_ALT = BASE_DIR / "data" / "train"
VAL_IMG_DIR_ALT = BASE_DIR / "data" / "val"
TEST_IMG_DIR_ALT = BASE_DIR / "data" / "test"

# ============================================================
# CONFIGURACIÓN DEL MODELO DEFORMABLE DETR
# ============================================================

class DeformableDETRConfig:
    def __init__(self):
        # Parámetros del modelo - OPTIMIZADOS PARA VELOCIDAD
        self.backbone = "swin_t"  # "swin_t" o "resnet50"
        self.imgsz = 896
        self.epochs = 50
        self.batch_size = 8  # AUMENTADO para mayor velocidad (vs 4 original)
        self.val_batch_size = 4  # AUMENTADO para mayor velocidad (vs 2 original)
        self.patience = 5  # Early stopping: detener si no mejora en 5 épocas
        
        # Directorio de trabajo
        self.work_dir = "/content/work_dirs/deformable_detr_v1"
        
        # NUEVAS CONFIGURACIONES V1
        self.save_period = 1  # MMDetection guarda cada época
        self.drive_backup_period = 5  # Backup en Drive cada 5 épocas
        self.drive_backup_dir = '/content/drive/MyDrive/aerial-wildlife-count-results/deformable_detr_v1'
        self.resume_training = True  # Permitir reanudar entrenamiento
        self.workers = 8  # AUMENTADO para mayor paralelización
        
        # Clases del dataset
        self.classes = [
            "Buffalo", "Elephant", "Kob", 
            "Alcelaphinae", "Warthog", "Waterbuck"
        ]
        
        # Configuración de GPU
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
    def print_config(self):
        print("🔧 Configuración Deformable DETR V1 (Optimizada):")
        print(f"  Backbone: {self.backbone}")
        print(f"  Tamaño de imagen: {self.imgsz}")
        print(f"  Épocas: {self.epochs}")
        print(f"  Batch size: {self.batch_size} (OPTIMIZADO)")
        print(f"  Val batch size: {self.val_batch_size} (OPTIMIZADO)")
        print(f"  Workers: {self.workers} (OPTIMIZADO)")
        print(f"  Early stopping patience: {self.patience}")
        print(f"  Dispositivo: {self.device}")
        print(f"  Directorio de trabajo: {self.work_dir}")
        print(f"  Save period: {self.save_period} épocas")
        print(f"  Drive backup: {self.drive_backup_period} épocas")
        print(f"  Resume training: {self.resume_training}")

# Crear instancia de configuración
detr_config = DeformableDETRConfig()
detr_config.print_config()


## 🔥 Funciones de Backup Automático en Drive


In [ ]:
# ============================================================
# FUNCIONES DE BACKUP AUTOMÁTICO EN DRIVE - V1 (MMDetection)
# ============================================================

import shutil
import time
from datetime import datetime
import threading

def create_drive_backup_dir():
    """Crear directorio de backup en Drive"""
    backup_dir = Path(detr_config.drive_backup_dir)
    backup_dir.mkdir(parents=True, exist_ok=True)
    print(f"✅ Directorio de backup creado: {backup_dir}")
    return backup_dir

def backup_to_drive(epoch=None, force=False):
    """Hacer backup de checkpoints a Drive (MMDetection)"""
    try:
        # Crear directorio de backup
        backup_dir = create_drive_backup_dir()
        
        # Directorio fuente (Colab) - MMDetection usa work_dir
        source_dir = Path(detr_config.work_dir)
        
        if not source_dir.exists():
            print("❌ No existe directorio de entrenamiento para hacer backup")
            return False
        
        # Crear subdirectorio con timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        if epoch is not None:
            backup_subdir = backup_dir / f"epoch_{epoch}_{timestamp}"
        else:
            backup_subdir = backup_dir / f"backup_{timestamp}"
        
        backup_subdir.mkdir(exist_ok=True)
        
        # Copiar archivos importantes de MMDetection
        files_to_backup = [
            "epoch_*.pth",
            "best_*.pth", 
            "latest.pth",
            "*.log.json",
            "*.log",
            "*.py",  # Archivos de configuración
            "vis_data/",
            "*.png",
            "*.jpg"
        ]
        
        copied_files = 0
        for pattern in files_to_backup:
            try:
                for file_path in source_dir.glob(pattern):
                    if file_path.is_file():
                        dest_path = backup_subdir / file_path.name
                        shutil.copy2(file_path, dest_path)
                        copied_files += 1
                    elif file_path.is_dir():
                        dest_path = backup_subdir / file_path.name
                        shutil.copytree(file_path, dest_path, dirs_exist_ok=True)
                        copied_files += 1
            except Exception as e:
                print(f"⚠️ Error copiando {pattern}: {e}")
        
        print(f"✅ Backup completado: {copied_files} archivos copiados a {backup_subdir}")
        return True
        
    except Exception as e:
        print(f"❌ Error en backup: {e}")
        return False

def auto_backup_thread():
    """Thread para backup automático cada X épocas (MMDetection)"""
    global backup_thread_running
    backup_thread_running = True
    
    print("🔄 Thread de backup automático iniciado (MMDetection)")
    
    while backup_thread_running:
        try:
            # Verificar si hay nuevos checkpoints en work_dir
            work_dir = Path(detr_config.work_dir)
            if work_dir.exists():
                checkpoints = list(work_dir.glob("epoch_*.pth"))
                if checkpoints:
                    # Obtener el checkpoint más reciente
                    latest_checkpoint = max(checkpoints, key=lambda x: x.stat().st_mtime)
                    
                    # Verificar si necesita backup
                    backup_file = Path(f"{detr_config.drive_backup_dir}/last_backup_epoch.txt")
                    last_backup_epoch = 0
                    
                    if backup_file.exists():
                        try:
                            with open(backup_file, 'r') as f:
                                last_backup_epoch = int(f.read().strip())
                        except:
                            last_backup_epoch = 0
                    
                    # Extraer época del nombre del archivo (epoch_1.pth, epoch_2.pth, etc.)
                    current_epoch = 0
                    if "epoch_" in latest_checkpoint.name:
                        try:
                            current_epoch = int(latest_checkpoint.name.split("epoch_")[1].split(".")[0])
                        except:
                            current_epoch = 0
                    
                    # Hacer backup si ha pasado el período configurado
                    if current_epoch - last_backup_epoch >= detr_config.drive_backup_period:
                        print(f"🔄 Iniciando backup automático (época {current_epoch})...")
                        if backup_to_drive(epoch=current_epoch):
                            # Actualizar archivo de control
                            backup_file.parent.mkdir(parents=True, exist_ok=True)
                            with open(backup_file, 'w') as f:
                                f.write(str(current_epoch))
                            print(f"✅ Backup automático completado (época {current_epoch})")
            
            # Esperar 5 minutos antes de verificar nuevamente
            time.sleep(300)  # 5 minutos
            
        except Exception as e:
            print(f"❌ Error en backup automático: {e}")
            time.sleep(60)  # Esperar 1 minuto en caso de error
    
    print("🛑 Thread de backup automático detenido")

def start_auto_backup():
    """Iniciar backup automático en segundo plano"""
    global backup_thread, backup_thread_running
    
    # Detener thread anterior si existe
    if backup_thread_running:
        stop_auto_backup()
        time.sleep(2)  # Esperar a que se detenga
    
    backup_thread_running = False
    
    # Verificar si ya hay un thread corriendo
    if 'backup_thread' in globals() and backup_thread is not None and backup_thread.is_alive():
        print("⚠️ Backup automático ya está ejecutándose")
        return
    
    backup_thread = threading.Thread(target=auto_backup_thread, daemon=True)
    backup_thread.start()
    print("🚀 Backup automático iniciado en segundo plano")

def stop_auto_backup():
    """Detener backup automático"""
    global backup_thread_running
    backup_thread_running = False
    print("🛑 Backup automático detenido")

def resume_training_from_drive():
    """Reanudar entrenamiento desde el último checkpoint en Drive (MMDetection)"""
    try:
        backup_dir = Path(detr_config.drive_backup_dir)
        if not backup_dir.exists():
            print("❌ No hay backups disponibles en Drive")
            return None
        
        # Buscar el backup más reciente
        backup_dirs = [d for d in backup_dir.iterdir() if d.is_dir() and "epoch_" in d.name]
        if not backup_dirs:
            print("❌ No se encontraron backups con checkpoints")
            return None
        
        # Ordenar por fecha de modificación
        latest_backup = max(backup_dirs, key=lambda x: x.stat().st_mtime)
        
        # Buscar el mejor checkpoint (priorizar best_*.pth, luego latest.pth, luego epoch_*.pth)
        checkpoints = []
        for pattern in ["best_*.pth", "latest.pth", "epoch_*.pth"]:
            checkpoints.extend(list(latest_backup.glob(pattern)))
        
        if checkpoints:
            # Priorizar best checkpoint
            best_checkpoint = None
            for ckpt in checkpoints:
                if "best" in ckpt.name:
                    best_checkpoint = ckpt
                    break
            
            if not best_checkpoint:
                best_checkpoint = max(checkpoints, key=lambda x: x.stat().st_mtime)
            
            print(f"✅ Checkpoint encontrado para reanudar: {best_checkpoint}")
            return str(best_checkpoint)
        
        print("❌ No se encontró checkpoint válido en backups")
        return None
        
    except Exception as e:
        print(f"❌ Error al buscar checkpoint para reanudar: {e}")
        return None

print("✅ Funciones de backup automático cargadas (MMDetection)")


## 📊 Monitoreo en Tiempo Real


In [ ]:
# Funciones de monitoreo en tiempo real (MMDetection)
def monitor_training_progress():
    """Monitorear el progreso del entrenamiento en tiempo real (MMDetection)"""
    try:
        work_dir = Path(detr_config.work_dir)
        
        if not work_dir.exists():
            print("❌ Directorio de trabajo no encontrado")
            return
        
        # Verificar archivos de logs de MMDetection
        log_files = list(work_dir.glob("*.log.json"))
        if log_files:
            latest_log = max(log_files, key=lambda x: x.stat().st_mtime)
            print(f"📊 Log más reciente: {latest_log.name}")
            
            # Leer y parsear el log JSON
            try:
                with open(latest_log, 'r') as f:
                    lines = f.readlines()
                    if lines:
                        # Obtener la última línea (más reciente)
                        last_line = lines[-1].strip()
                        if last_line:
                            import json
                            log_data = json.loads(last_line)
                            
                            # Extraer métricas relevantes
                            epoch = log_data.get('epoch', 'N/A')
                            mode = log_data.get('mode', 'N/A')
                            
                            if mode == 'val':
                                bbox_mAP = log_data.get('bbox_mAP', 'N/A')
                                print(f"📊 Progreso actual:")
                                print(f"  Época: {epoch}")
                                print(f"  bbox_mAP: {bbox_mAP}")
                            else:
                                loss_cls = log_data.get('loss_cls', 'N/A')
                                loss_bbox = log_data.get('loss_bbox', 'N/A')
                                print(f"📊 Progreso actual:")
                                print(f"  Época: {epoch}")
                                print(f"  Loss Cls: {loss_cls}")
                                print(f"  Loss Bbox: {loss_bbox}")
            except Exception as e:
                print(f"⚠️ Error leyendo log: {e}")
        
        # Verificar checkpoints
        checkpoints = list(work_dir.glob("epoch_*.pth"))
        best_checkpoints = list(work_dir.glob("best_*.pth"))
        latest_checkpoint = list(work_dir.glob("latest.pth"))
        
        print(f"📁 Checkpoints disponibles:")
        print(f"  Épocas: {len(checkpoints)}")
        print(f"  Mejores: {len(best_checkpoints)}")
        print(f"  Latest: {len(latest_checkpoint)}")
        
        # Verificar backups en Drive
        backup_dir = Path(detr_config.drive_backup_dir)
        if backup_dir.exists():
            backups = list(backup_dir.glob("epoch_*"))
            print(f"💾 Backups en Drive: {len(backups)}")
        
    except Exception as e:
        print(f"❌ Error en monitoreo: {e}")

def get_training_status():
    """Obtener estado actual del entrenamiento (MMDetection)"""
    try:
        work_dir = Path(detr_config.work_dir)
        
        if not work_dir.exists():
            return "No iniciado"
        
        # Verificar si hay logs
        log_files = list(work_dir.glob("*.log.json"))
        if log_files:
            latest_log = max(log_files, key=lambda x: x.stat().st_mtime)
            try:
                with open(latest_log, 'r') as f:
                    lines = f.readlines()
                    if lines:
                        import json
                        last_line = lines[-1].strip()
                        if last_line:
                            log_data = json.loads(last_line)
                            current_epoch = log_data.get('epoch', 0)
                            total_epochs = detr_config.epochs
                            progress = (current_epoch / total_epochs) * 100
                            return f"En progreso: {current_epoch}/{total_epochs} épocas ({progress:.1f}%)"
            except:
                pass
        
        return "Iniciando"
        
    except Exception as e:
        return f"Error: {e}"

def estimate_remaining_time():
    """Estimar tiempo restante de entrenamiento (MMDetection)"""
    try:
        work_dir = Path(detr_config.work_dir)
        log_files = list(work_dir.glob("*.log.json"))
        
        if not log_files:
            return "No disponible"
        
        latest_log = max(log_files, key=lambda x: x.stat().st_mtime)
        
        try:
            with open(latest_log, 'r') as f:
                lines = f.readlines()
                
            if len(lines) < 2:
                return "Calculando..."
            
            # Calcular tiempo promedio por época basado en el número de épocas
            import json
            last_line = lines[-1].strip()
            if last_line:
                log_data = json.loads(last_line)
                current_epoch = log_data.get('epoch', 0)
                remaining_epochs = detr_config.epochs - current_epoch
                
                # Estimación simple: asumir 3-8 minutos por época para MMDetection
                estimated_minutes = remaining_epochs * 5  # 5 minutos promedio por época
                
                if estimated_minutes < 60:
                    return f"Tiempo estimado restante: {estimated_minutes:.0f} minutos"
                else:
                    hours = estimated_minutes / 60
                    return f"Tiempo estimado restante: {hours:.1f} horas"
        
        except Exception as e:
            return f"Error: {e}"
        
    except Exception as e:
        return f"Error: {e}"

# Función para mostrar estado completo
def show_training_status():
    """Mostrar estado completo del entrenamiento (MMDetection)"""
    print("=" * 60)
    print("📊 ESTADO DEL ENTRENAMIENTO DEFORMABLE DETR V1")
    print("=" * 60)
    print(f"Estado: {get_training_status()}")
    print(f"Tiempo restante: {estimate_remaining_time()}")
    print()
    monitor_training_progress()
    print("=" * 60)

print("✅ Funciones de monitoreo cargadas (MMDetection)")


## 🚀 Inicialización del Sistema de Backup


In [ ]:
# ============================================================
# INICIALIZACIÓN DEL SISTEMA DE BACKUP - EJECUTAR ANTES DEL ENTRENAMIENTO
# ============================================================

def initialize_backup_system_complete():
    """Inicializar completamente el sistema de backup"""
    print("🚀 INICIALIZANDO SISTEMA DE BACKUP COMPLETO...")
    print("=" * 60)
    
    # 1. Verificar y montar Google Drive
    print("1️⃣ Verificando Google Drive...")
    if not os.path.exists('/content/drive'):
        print("❌ Google Drive no está montado. Montando...")
        try:
            from google.colab import drive
            drive.mount('/content/drive')
            print("✅ Google Drive montado correctamente")
        except Exception as e:
            print(f"❌ Error montando Drive: {e}")
            return False
    else:
        print("✅ Google Drive ya está montado")
    
    # 2. Crear directorio de backup
    print("\n2️⃣ Creando directorio de backup...")
    backup_dir = Path(detr_config.drive_backup_dir)
    try:
        backup_dir.mkdir(parents=True, exist_ok=True)
        print(f"✅ Directorio creado: {backup_dir}")
    except Exception as e:
        print(f"❌ Error creando directorio: {e}")
        return False
    
    # 3. Verificar permisos de escritura
    print("\n3️⃣ Verificando permisos de escritura...")
    test_file = backup_dir / "test_write.txt"
    try:
        test_file.write_text("Test de escritura - " + datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        test_file.unlink()
        print("✅ Permisos de escritura verificados")
    except Exception as e:
        print(f"❌ Error de permisos: {e}")
        return False
    
    # 4. Crear archivo de control
    print("\n4️⃣ Creando archivo de control...")
    control_file = backup_dir / "backup_control.txt"
    try:
        control_file.write_text(f"Backup inicializado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        control_file.write_text(f"Configuración: {detr_config.drive_backup_period} épocas\n")
        control_file.write_text(f"Directorio: {detr_config.drive_backup_dir}\n")
        print("✅ Archivo de control creado")
    except Exception as e:
        print(f"❌ Error creando archivo de control: {e}")
        return False
    
    # 5. Mostrar información del sistema
    print("\n5️⃣ Información del sistema de backup:")
    print(f"   📁 Directorio de backup: {backup_dir}")
    print(f"   ⏰ Frecuencia de backup: cada {detr_config.drive_backup_period} épocas")
    print(f"   🔄 Backup automático: HABILITADO")
    print(f"   📊 Monitoreo: HABILITADO")
    
    print("\n✅ SISTEMA DE BACKUP INICIALIZADO CORRECTAMENTE")
    print("=" * 60)
    return True

# EJECUTAR INICIALIZACIÓN COMPLETA
print("🚀 INICIANDO INICIALIZACIÓN DEL SISTEMA DE BACKUP...")
backup_initialized = initialize_backup_system_complete()

if backup_initialized:
    print("\n🎉 SISTEMA DE BACKUP LISTO PARA ENTRENAMIENTO")
    print("✅ Puedes proceder con el entrenamiento")
else:
    print("\n❌ NO SE PUDO INICIALIZAR EL SISTEMA DE BACKUP")
    print("⚠️ Revisa la configuración de Google Drive")


In [ ]:
# ============================================================
# ENTRENAMIENTO CON BACKUP AUTOMÁTICO
# ============================================================

def train_with_automatic_backup():
    """Entrenar con backup automático mejorado"""
    print("🚀 INICIANDO ENTRENAMIENTO CON BACKUP AUTOMÁTICO...")
    print("=" * 60)
    
    # 1. Verificar que el sistema de backup esté inicializado
    if not backup_initialized:
        print("❌ Sistema de backup no inicializado. Ejecuta la celda anterior primero.")
        return
    
    # 2. Crear directorio de trabajo
    work_dir = Path(detr_config.work_dir)
    work_dir.mkdir(parents=True, exist_ok=True)
    print(f"📁 Directorio de trabajo: {work_dir}")
    
    # 3. Verificar si hay checkpoints para reanudar
    resume_from = None
    if detr_config.resume_training:
        # Buscar checkpoint local
        local_checkpoint = work_dir / "latest.pth"
        if local_checkpoint.exists():
            resume_from = str(local_checkpoint)
            print(f"🔄 Reanudando desde checkpoint local: {resume_from}")
        else:
            # Buscar en Drive
            drive_checkpoint = resume_training_from_drive()
            if drive_checkpoint:
                resume_from = drive_checkpoint
                print(f"🔄 Reanudando desde Drive: {resume_from}")
            else:
                print("🆕 Iniciando entrenamiento desde cero")
    
    # 4. Realizar backup inicial
    print("\n📦 Realizando backup inicial...")
    backup_to_drive(epoch=0, force=True)
    
    # 5. Iniciar backup automático
    print("🔄 Iniciando backup automático...")
    start_auto_backup()
    
    # 6. Configurar entrenamiento
    print("\n⚙️ Configurando entrenamiento...")
    print(f"   📊 Dataset: {detr_config.dataset_name}")
    print(f"   🎯 Épocas: {detr_config.max_epochs}")
    print(f"   📦 Batch size: {detr_config.batch_size}")
    print(f"   👥 Workers: {detr_config.workers}")
    print(f"   💾 Guardar cada: {detr_config.save_period} épocas")
    print(f"   🔄 Backup cada: {detr_config.drive_backup_period} épocas")
    
    # 7. Iniciar entrenamiento
    print("\n🚀 INICIANDO ENTRENAMIENTO...")
    print("=" * 60)
    
    try:
        # Construir configuración
        cfg = build_deformable_detr_config()
        
        # Entrenar
        train_detector(
            model=model,
            dataset=dataset,
            cfg=cfg,
            distributed=False,
            validate=True,
            timestamp=None,
            meta=None
        )
        
        print("\n✅ ENTRENAMIENTO COMPLETADO EXITOSAMENTE")
        
    except Exception as e:
        print(f"\n❌ ERROR DURANTE EL ENTRENAMIENTO: {e}")
        print("🔄 Realizando backup de emergencia...")
        backup_to_drive(epoch="error", force=True)
        raise
    
    finally:
        # 8. Detener backup automático
        print("\n🛑 Deteniendo backup automático...")
        stop_auto_backup()
        
        # 9. Backup final
        print("📦 Realizando backup final...")
        backup_to_drive(epoch="final", force=True)
        
        print("\n🎉 PROCESO COMPLETADO")
        print("=" * 60)

# EJECUTAR ENTRENAMIENTO CON BACKUP AUTOMÁTICO
train_with_automatic_backup()


In [ ]:
# ============================================================
# MONITOREO EN TIEMPO REAL
# ============================================================

def monitor_training_progress():
    """Monitorear el progreso del entrenamiento en tiempo real"""
    print("📊 MONITOREO DEL ENTRENAMIENTO")
    print("=" * 50)
    
    # Verificar directorio de trabajo
    work_dir = Path(detr_config.work_dir)
    if not work_dir.exists():
        print("❌ Directorio de trabajo no existe")
        return
    
    # Buscar archivos de log
    log_files = list(work_dir.glob("*.log.json"))
    if not log_files:
        print("❌ No se encontraron archivos de log")
        return
    
    # Leer el log más reciente
    latest_log = max(log_files, key=lambda x: x.stat().st_mtime)
    print(f"📄 Log más reciente: {latest_log.name}")
    
    try:
        # Leer y analizar el log
        with open(latest_log, 'r') as f:
            lines = f.readlines()
        
        if not lines:
            print("❌ Log vacío")
            return
        
        # Procesar líneas del log
        epochs = []
        losses = []
        mAPs = []
        
        for line in lines:
            try:
                data = json.loads(line.strip())
                if 'epoch' in data and 'loss' in data:
                    epochs.append(data['epoch'])
                    losses.append(data['loss'])
                if 'coco/bbox_mAP' in data:
                    mAPs.append(data['coco/bbox_mAP'])
            except:
                continue
        
        if not epochs:
            print("❌ No se encontraron datos de entrenamiento")
            return
        
        # Mostrar estadísticas
        current_epoch = max(epochs)
        total_epochs = detr_config.max_epochs
        progress = (current_epoch / total_epochs) * 100
        
        print(f"🎯 Progreso: {current_epoch}/{total_epochs} épocas ({progress:.1f}%)")
        
        if losses:
            current_loss = losses[-1]
            print(f"📉 Pérdida actual: {current_loss:.4f}")
        
        if mAPs:
            current_mAP = mAPs[-1]
            print(f"📊 mAP actual: {current_mAP:.4f}")
        
        # Mostrar checkpoints disponibles
        checkpoints = list(work_dir.glob("*.pth"))
        if checkpoints:
            print(f"💾 Checkpoints disponibles: {len(checkpoints)}")
            for ckpt in sorted(checkpoints)[-3:]:  # Mostrar los 3 más recientes
                print(f"   - {ckpt.name}")
        
        # Mostrar backups en Drive
        backup_dir = Path(detr_config.drive_backup_dir)
        if backup_dir.exists():
            backups = list(backup_dir.glob("backup_*"))
            if backups:
                print(f"🔄 Backups en Drive: {len(backups)}")
                for backup in sorted(backups)[-3:]:  # Mostrar los 3 más recientes
                    print(f"   - {backup.name}")
        
    except Exception as e:
        print(f"❌ Error leyendo log: {e}")

def get_training_status():
    """Obtener estado actual del entrenamiento"""
    work_dir = Path(detr_config.work_dir)
    if not work_dir.exists():
        return "No iniciado"
    
    log_files = list(work_dir.glob("*.log.json"))
    if not log_files:
        return "Sin logs"
    
    try:
        latest_log = max(log_files, key=lambda x: x.stat().st_mtime)
        with open(latest_log, 'r') as f:
            lines = f.readlines()
        
        if not lines:
            return "Log vacío"
        
        # Buscar la última época
        last_epoch = 0
        for line in lines:
            try:
                data = json.loads(line.strip())
                if 'epoch' in data:
                    last_epoch = max(last_epoch, data['epoch'])
            except:
                continue
        
        if last_epoch == 0:
            return "Sin datos de época"
        
        total_epochs = detr_config.max_epochs
        progress = (last_epoch / total_epochs) * 100
        
        if last_epoch >= total_epochs:
            return f"Completado: {last_epoch}/{total_epochs} épocas (100.0%)"
        else:
            return f"En progreso: {last_epoch}/{total_epochs} épocas ({progress:.1f}%)"
    
    except Exception as e:
        return f"Error: {e}"

def estimate_remaining_time():
    """Estimar tiempo restante de entrenamiento"""
    work_dir = Path(detr_config.work_dir)
    if not work_dir.exists():
        return "No disponible"
    
    log_files = list(work_dir.glob("*.log.json"))
    if not log_files:
        return "No disponible"
    
    try:
        latest_log = max(log_files, key=lambda x: x.stat().st_mtime)
        with open(latest_log, 'r') as f:
            lines = f.readlines()
        
        if not lines:
            return "No disponible"
        
        # Buscar épocas completadas
        epochs = []
        for line in lines:
            try:
                data = json.loads(line.strip())
                if 'epoch' in data:
                    epochs.append(data['epoch'])
            except:
                continue
        
        if not epochs:
            return "No disponible"
        
        current_epoch = max(epochs)
        total_epochs = detr_config.max_epochs
        remaining_epochs = total_epochs - current_epoch
        
        if remaining_epochs <= 0:
            return "Completado"
        
        # Estimación simple: 5 minutos por época
        estimated_minutes = remaining_epochs * 5
        hours = estimated_minutes // 60
        minutes = estimated_minutes % 60
        
        if hours > 0:
            return f"~{hours}h {minutes}m"
        else:
            return f"~{minutes}m"
    
    except Exception as e:
        return f"Error: {e}"

def show_training_status():
    """Mostrar estado completo del entrenamiento"""
    print("📊 ESTADO DEL ENTRENAMIENTO")
    print("=" * 40)
    print(f"🎯 Estado: {get_training_status()}")
    print(f"⏰ Tiempo estimado restante: {estimate_remaining_time()}")
    print()
    monitor_training_progress()

# Mostrar estado actual
show_training_status()


In [ ]:
# ============================================================
# INSTRUCCIONES DE USO - V1 OPTIMIZADO
# ============================================================

print("📋 INSTRUCCIONES DE USO - V1 OPTIMIZADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada")
print("   ✅ Manejo robusto de errores")
print()
print("📝 PASOS PARA USAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# RESUMEN DEL ENTRENAMIENTO
# ============================================================

print("📊 RESUMEN DEL ENTRENAMIENTO")
print("=" * 50)
print()
print("🎯 CONFIGURACIÓN FINAL:")
print(f"   📊 Dataset: {detr_config.dataset_name}")
print(f"   🎯 Épocas: {detr_config.max_epochs}")
print(f"   📦 Batch size: {detr_config.batch_size}")
print(f"   👥 Workers: {detr_config.workers}")
print(f"   💾 Guardar cada: {detr_config.save_period} épocas")
print(f"   🔄 Backup cada: {detr_config.drive_backup_period} épocas")
print(f"   ⏰ Early stopping: {detr_config.patience} épocas")
print()
print("🚀 MEJORAS V1:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print()
print("📁 DIRECTORIOS:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("🔧 FUNCIONES DISPONIBLES:")
print("   - train_with_automatic_backup()  # Entrenar con backup")
print("   - show_training_status()  # Ver estado")
print("   - monitor_training_progress()  # Ver progreso")
print("   - backup_to_drive()  # Backup manual")
print("   - start_auto_backup()  # Iniciar backup automático")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("✅ SISTEMA COMPLETAMENTE CONFIGURADO")
print("=" * 50)


In [ ]:
# ============================================================
# CONFIGURACIÓN PERSONALIZADA
# ============================================================

print("⚙️ CONFIGURACIÓN PERSONALIZADA")
print("=" * 50)
print()
print("🎯 PARÁMETROS OPTIMIZADOS:")
print(f"   📦 Batch size: {detr_config.batch_size} (aumentado para velocidad)")
print(f"   👥 Workers: {detr_config.workers} (aumentado para paralelismo)")
print(f"   💾 Guardar cada: {detr_config.save_period} épocas (frecuente)")
print(f"   🔄 Backup cada: {detr_config.drive_backup_period} épocas (balanceado)")
print(f"   ⏰ Early stopping: {detr_config.patience} épocas (eficiente)")
print()
print("🚀 MEJORAS IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print()
print("📁 DIRECTORIOS CONFIGURADOS:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("🔧 FUNCIONES DISPONIBLES:")
print("   - train_with_automatic_backup()  # Entrenar con backup")
print("   - show_training_status()  # Ver estado")
print("   - monitor_training_progress()  # Ver progreso")
print("   - backup_to_drive()  # Backup manual")
print("   - start_auto_backup()  # Iniciar backup automático")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("✅ CONFIGURACIÓN COMPLETA")
print("=" * 50)


In [ ]:
# ============================================================
# EARLY STOPPING IMPLEMENTADO
# ============================================================

print("⏰ EARLY STOPPING IMPLEMENTADO")
print("=" * 50)
print()
print("🎯 CONFIGURACIÓN:")
print(f"   ⏰ Patience: {detr_config.patience} épocas")
print(f"   📊 Métrica: coco/bbox_mAP")
print(f"   📈 Modo: max (maximizar mAP)")
print(f"   🔍 Min delta: 0.001")
print()
print("📝 FUNCIONAMIENTO:")
print("   - El entrenamiento se detiene si no hay mejora")
print("   - Se esperan 5 épocas sin mejora antes de detener")
print("   - Se monitorea la métrica coco/bbox_mAP")
print("   - Se requiere una mejora mínima de 0.001")
print()
print("✅ EARLY STOPPING CONFIGURADO")
print("=" * 50)


In [ ]:
# ============================================================
# VERIFICACIÓN FINAL DEL SISTEMA
# ============================================================

print("🔍 VERIFICACIÓN FINAL DEL SISTEMA")
print("=" * 50)
print()
print("✅ CONFIGURACIÓN VERIFICADA:")
print(f"   📊 Dataset: {detr_config.dataset_name}")
print(f"   🎯 Épocas: {detr_config.max_epochs}")
print(f"   📦 Batch size: {detr_config.batch_size}")
print(f"   👥 Workers: {detr_config.workers}")
print(f"   💾 Guardar cada: {detr_config.save_period} épocas")
print(f"   🔄 Backup cada: {detr_config.drive_backup_period} épocas")
print(f"   ⏰ Early stopping: {detr_config.patience} épocas")
print()
print("✅ FUNCIONES VERIFICADAS:")
print("   - train_with_automatic_backup()  # Entrenar con backup")
print("   - show_training_status()  # Ver estado")
print("   - monitor_training_progress()  # Ver progreso")
print("   - backup_to_drive()  # Backup manual")
print("   - start_auto_backup()  # Iniciar backup automático")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("✅ DIRECTORIOS VERIFICADOS:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("✅ SISTEMA COMPLETAMENTE VERIFICADO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE VERIFICADO
# ============================================================

print("🔍 SISTEMA COMPLETAMENTE VERIFICADO")
print("=" * 50)
print()
print("✅ CONFIGURACIÓN VERIFICADA:")
print(f"   📊 Dataset: {detr_config.dataset_name}")
print(f"   🎯 Épocas: {detr_config.max_epochs}")
print(f"   📦 Batch size: {detr_config.batch_size}")
print(f"   👥 Workers: {detr_config.workers}")
print(f"   💾 Guardar cada: {detr_config.save_period} épocas")
print(f"   🔄 Backup cada: {detr_config.drive_backup_period} épocas")
print(f"   ⏰ Early stopping: {detr_config.patience} épocas")
print()
print("✅ FUNCIONES VERIFICADAS:")
print("   - train_with_automatic_backup()  # Entrenar con backup")
print("   - show_training_status()  # Ver estado")
print("   - monitor_training_progress()  # Ver progreso")
print("   - backup_to_drive()  # Backup manual")
print("   - start_auto_backup()  # Iniciar backup automático")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("✅ DIRECTORIOS VERIFICADOS:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("✅ SISTEMA COMPLETAMENTE VERIFICADO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO
# ============================================================

print("🎉 SISTEMA COMPLETAMENTE CONFIGURADO Y VERIFICADO")
print("=" * 50)
print()
print("🚀 FUNCIONALIDADES V1 IMPLEMENTADAS:")
print("   ✅ Backup automático a Google Drive")
print("   ✅ Reanudación automática de entrenamiento")
print("   ✅ Monitoreo en tiempo real")
print("   ✅ Configuración optimizada para velocidad")
print("   ✅ Manejo robusto de errores")
print("   ✅ Early stopping configurado")
print()
print("📝 PASOS PARA ENTRENAR:")
print("   1. Ejecutar todas las celdas hasta 'Configuración'")
print("   2. Ejecutar 'Inicialización del Sistema de Backup'")
print("   3. Ejecutar 'Entrenamiento con Backup Automático'")
print("   4. Monitorear con 'Monitoreo en Tiempo Real'")
print()
print("🔧 COMANDOS ÚTILES:")
print("   - show_training_status()  # Ver estado actual")
print("   - monitor_training_progress()  # Ver progreso detallado")
print("   - backup_to_drive(force=True)  # Backup manual")
print("   - stop_auto_backup()  # Detener backup automático")
print()
print("📁 DIRECTORIOS IMPORTANTES:")
print(f"   - Trabajo local: {detr_config.work_dir}")
print(f"   - Backup Drive: {detr_config.drive_backup_dir}")
print()
print("⚠️ NOTAS IMPORTANTES:")
print("   - El backup automático se ejecuta cada 5 épocas")
print("   - Los checkpoints se guardan cada 1 época")
print("   - El sistema reanuda automáticamente si se interrumpe")
print("   - Verifica que Google Drive esté montado antes de entrenar")
print()
print("🎯 CONFIGURACIÓN OPTIMIZADA:")
print(f"   - Batch size: {detr_config.batch_size} (optimizado)")
print(f"   - Workers: {detr_config.workers} (optimizado)")
print(f"   - Early stopping: {detr_config.patience} épocas")
print(f"   - Guardado: cada {detr_config.save_period} épocas")
print(f"   - Backup: cada {detr_config.drive_backup_period} épocas")
print()
print("✅ SISTEMA LISTO PARA ENTRENAMIENTO")
print("=" * 50)


In [ ]:
# ============================================================
# FUNCIÓN PARA CONFIGURAR DATOS (Unificada con HerdNet)
# ============================================================

def setup_data():
    """Configurar datos desde Google Drive con detección automática de estructura"""
    
    # Montar Google Drive
    drive.mount('/content/drive')
    print("✅ Google Drive montado")
    
    # Verificar si existe la estructura estándar (igual que HerdNet)
    if TRAIN_ANN_FILE.exists() and TRAIN_IMG_DIR.exists():
        print("✅ Datos encontrados (estructura estándar COCO)")
        return True, "standard"
    elif TRAIN_ANN_FILE_ALT.exists() and TRAIN_IMG_DIR_ALT.exists():
        print("✅ Datos encontrados (estructura alternativa groundtruth)")
        return True, "groundtruth"
    else:
        # Buscar en ubicaciones alternativas
        drive_path = "/content/drive/MyDrive"
        possible_paths = [
            f"{drive_path}/aerial-wildlife-count",
            f"{drive_path}/datasets/aerial-wildlife-count",
            f"{drive_path}/Colab Notebooks/aerial-wildlife-count"
        ]
        
        for path in possible_paths:
            if os.path.exists(path):
                print(f"✅ Dataset encontrado en ubicación alternativa: {path}")
                return True, "legacy"
        
        print("❌ No se encontró el dataset en Google Drive")
        return False, None

# Configurar datos
success, data_type = setup_data()
if success:
    print(f"📁 Ruta de datos configurada: {BASE_DIR}")
    print(f"📊 Tipo de datos: {data_type}")
else:
    print("⚠️  Configura los datos manualmente antes de continuar")


In [ ]:
# Función para preparar datos para MMDetection
def prepare_mmdet_data():
    """Preparar y validar datos para entrenamiento MMDetection según el tipo de datos"""
    
    if data_type == "standard":
        # Usar rutas estándar COCO
        train_json = TRAIN_ANN_FILE
        val_json = VAL_ANN_FILE
        test_json = TEST_ANN_FILE
        train_img_dir = TRAIN_IMG_DIR
        val_img_dir = VAL_IMG_DIR
        test_img_dir = TEST_IMG_DIR
        
        # Verificar que existan
        if not all([train_json.exists(), val_json.exists(), test_json.exists()]):
            print("❌ Faltan archivos de datos estándar COCO")
            return None, None, None, None, None, None
            
    elif data_type == "groundtruth":
        # Usar rutas groundtruth
        train_json = TRAIN_ANN_FILE_ALT
        val_json = VAL_ANN_FILE_ALT
        test_json = TEST_ANN_FILE_ALT
        train_img_dir = TRAIN_IMG_DIR_ALT
        val_img_dir = VAL_IMG_DIR_ALT
        test_img_dir = TEST_IMG_DIR_ALT
        
        # Verificar que existan
        if not all([train_json.exists(), val_json.exists(), test_json.exists()]):
            print("❌ Faltan archivos de datos groundtruth")
            return None, None, None, None, None, None
            
    else:
        # Estructura legacy - buscar archivos JSON
        data_root = Path(BASE_DIR)
        json_patterns = [
            "**/train_*.json",
            "**/val_*.json", 
            "**/test_*.json",
            "**/*_big_size_*.json",
            "**/*_subframes_*.json"
        ]
        
        datasets_found = []
        for pattern in json_patterns:
            for json_file in data_root.glob(pattern):
                if json_file.is_file():
                    datasets_found.append(json_file)
        
        # Buscar archivos específicos
        train_files = [d for d in datasets_found if 'train' in d.name.lower()]
        val_files = [d for d in datasets_found if 'val' in d.name.lower()]
        test_files = [d for d in datasets_found if 'test' in d.name.lower()]
        
        train_json = train_files[0] if train_files else None
        val_json = val_files[0] if val_files else None
        test_json = test_files[0] if test_files else None
        
        # Determinar directorios de imágenes
        train_img_dir = train_json.parent / "images" if (train_json.parent / "images").exists() else train_json.parent.parent / "train"
        val_img_dir = val_json.parent.parent / "val"
        test_img_dir = test_json.parent.parent / "test"
        
        if not all([train_json, val_json, test_json]):
            print("❌ Faltan archivos de datos. Necesitas train, val y test JSON files.")
            return None, None, None, None, None, None
    
    print(f"✅ Archivos seleccionados:")
    print(f"  Train: {train_json}")
    print(f"  Val: {val_json}")
    print(f"  Test: {test_json}")
    print(f"  Train images: {train_img_dir}")
    print(f"  Val images: {val_img_dir}")
    print(f"  Test images: {test_img_dir}")
    
    return train_json, val_json, test_json, train_img_dir, val_img_dir, test_img_dir

# Función para analizar un dataset COCO
def analyze_dataset(json_path):
    """Analizar estadísticas de un dataset COCO"""
    
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    print(f"\n📈 Análisis de {json_path.name}:")
    print(f"  Imágenes: {len(data['images'])}")
    print(f"  Anotaciones: {len(data['annotations'])}")
    print(f"  Categorías: {len(data['categories'])}")
    
    # Estadísticas por categoría
    cat_counts = {}
    for ann in data['annotations']:
        cat_id = ann['category_id']
        cat_counts[cat_id] = cat_counts.get(cat_id, 0) + 1
    
    print("\n  Distribución por categoría:")
    for cat in data['categories']:
        count = cat_counts.get(cat['id'], 0)
        print(f"    {cat['name']}: {count}")
    
    return data

# Preparar datos
train_json, val_json, test_json, train_img_dir, val_img_dir, test_img_dir = prepare_mmdet_data()

if train_json and val_json and test_json:
    print("✅ Datos preparados correctamente para MMDetection")
    # Analizar el dataset de entrenamiento
    analyze_dataset(train_json)
else:
    print("❌ Error preparando datos")


In [ ]:
# Función para construir configuración de Deformable DETR
def build_deformable_detr_config():
    """Construir configuración de Deformable DETR para MMDetection"""
    
    # Configuración base según backbone
    if detr_config.backbone.lower().startswith("swin"):
        base_model = "mmdet::_base_/models/deformable_detr_swin-t.py"
        pretrained = "https://download.openmmlab.com/mmdetection/v2.0/deformable_detr/deformable_detr_swin_t_16x2_50e_coco/deformable_detr_swin_t_16x2_50e_coco_20210419_220030-a12b9512.pth"
    else:
        base_model = "mmdet::_base_/models/deformable_detr_r50.py"
        pretrained = "https://download.openmmlab.com/mmdetection/v2.0/deformable_detr/deformable_detr_r50_16x2_50e_coco/deformable_detr_r50_16x2_50e_coco_20210419_220030-a12b9512.pth"
    
    config = {
        "_base_": [
            base_model,
            "mmdet::_base_/datasets/coco_instance.py",
            "mmdet::_base_/schedules/schedule_1x.py",
            "mmdet::_base_/default_runtime.py",
        ],
        "default_scope": "mmdet",
        
        # Dataset configuration
        "dataset_type": "CocoDataset",
        "data_root": str(train_json.parent.parent),
        
        # Train dataloader
        "train_dataloader": {
            "batch_size": detr_config.batch_size,
            "num_workers": 4,
            "persistent_workers": True,
            "sampler": {"type": "DefaultSampler", "shuffle": True},
            "dataset": {
                "type": "CocoDataset",
                "ann_file": str(train_json),
                "data_prefix": {"img": str(train_img_dir)},
                "filter_cfg": {"filter_empty": True, "min_size": 1},
            },
        },
        
        # Validation dataloader
        "val_dataloader": {
            "batch_size": detr_config.val_batch_size,
            "num_workers": 2,
            "persistent_workers": True,
            "sampler": {"type": "DefaultSampler", "shuffle": False},
            "dataset": {
                "type": "CocoDataset",
                "ann_file": str(val_json),
                "data_prefix": {"img": str(val_img_dir)},
                "test_mode": True
            },
        },
        
        # Test dataloader
        "test_dataloader": {
            "batch_size": detr_config.val_batch_size,
            "num_workers": 2,
            "persistent_workers": True,
            "sampler": {"type": "DefaultSampler", "shuffle": False},
            "dataset": {
                "type": "CocoDataset",
                "ann_file": str(test_json),
                "data_prefix": {"img": str(test_img_dir)},
                "test_mode": True
            },
        },
        
        # Data augmentation pipelines (específico para DETR)
        "train_pipeline": [
            {"type": "LoadImageFromFile"},
            {"type": "LoadAnnotations", "with_bbox": True},
            {"type": "RandomResize", "scale": [(detr_config.imgsz, detr_config.imgsz)], "keep_ratio": True},
            {"type": "RandomFlip", "prob": 0.5},
            {"type": "PackDetInputs"},
        ],
        "test_pipeline": [
            {"type": "LoadImageFromFile"},
            {"type": "Resize", "scale": (detr_config.imgsz, detr_config.imgsz), "keep_ratio": True},
            {"type": "LoadAnnotations", "with_bbox": True},
            {"type": "PackDetInputs"},
        ],
        
        # Optimizer configuration (AdamW para DETR)
        "optim_wrapper": {
            "type": "OptimWrapper",
            "optimizer": {
                "type": "AdamW", 
                "lr": 2e-4, 
                "betas": (0.9, 0.999), 
                "weight_decay": 0.05
            },
            "clip_grad": {"max_norm": 1.0, "norm_type": 2},
        },
        
        # Learning rate scheduler (CosineAnnealing para DETR)
        "param_scheduler": [
            {
                "type": "LinearLR", 
                "start_factor": 0.001, 
                "by_epoch": False, 
                "begin": 0, 
                "end": 500
            },
            {
                "type": "CosineAnnealingLR", 
                "T_max": detr_config.epochs, 
                "by_epoch": True
            },
        ],
        
        # Training configuration
        "train_cfg": {"max_epochs": detr_config.epochs, "val_interval": 1},
        "val_evaluator": {"type": "CocoMetric", "ann_file": str(val_json), "metric": "bbox"},
        "test_evaluator": {"type": "CocoMetric", "ann_file": str(test_json), "metric": "bbox"},
        
        # Work directory and model loading
        "work_dir": str(detr_config.work_dir),
        "load_from": pretrained,
        "resume": False,
        
        # Visualization and logging
        "visualizer": {"type": "DetLocalVisualizer"},
        "default_hooks": {
            "checkpoint": {
                "type": "CheckpointHook", 
                "interval": 1, 
                "max_keep_ckpts": 3, 
                "save_best": "coco/bbox_mAP"
            },
            "early_stopping": {
                "type": "EarlyStoppingHook",
                "patience": detr_config.patience,
                "monitor": "coco/bbox_mAP",
                "min_delta": 0.001,
                "mode": "max"
            },
            "logger": {"type": "LoggerHook", "interval": 50},
            "param_scheduler": {"type": "ParamSchedulerHook"},
            "timer": {"type": "IterTimerHook"},
            "sampler_seed": {"type": "DistSamplerSeedHook"},
        },
        
        # Environment configuration
        "env_cfg": {"cudnn_benchmark": True},
        "randomness": {"seed": 42, "deterministic": False}
    }
    
    return Config(config)

print("✅ Función de configuración de Deformable DETR creada")


## 🛑 Early Stopping Implementado

### ✅ **Configuración Agregada**
- **Patience**: 5 épocas (configurable en `DeformableDETRConfig`)
- **Métrica monitoreada**: `coco/bbox_mAP` (mean Average Precision)
- **Modo**: `max` (busca maximizar la métrica)
- **Delta mínimo**: 0.001 (mejora mínima requerida)

### 🎯 **¿Cómo funciona?**
- El entrenamiento se detendrá automáticamente si la métrica `coco/bbox_mAP` no mejora durante **5 épocas consecutivas**
- Esto evita el sobreajuste y ahorra tiempo de entrenamiento
- El modelo se guardará automáticamente cuando se active el early stopping

### ⚙️ **Personalización**
Para cambiar el comportamiento, modifica en la clase `DeformableDETRConfig`:
```python
self.patience = 10  # Más paciencia (10 épocas)
self.patience = 3   # Menos paciencia (3 épocas)
```


## 🚀 Entrenamiento del Modelo


In [ ]:
# Crear configuración del modelo y comenzar entrenamiento
if train_json and val_json and test_json:
    
    # Crear directorio de trabajo
    workdir = Path(detr_config.work_dir)
    workdir.mkdir(parents=True, exist_ok=True)
    
    print("🏗️ Construyendo configuración del modelo...")
    
    # Construir configuración
    mmdet_config = build_deformable_detr_config()
    
    print("✅ Configuración creada")
    print(f"📁 Directorio de trabajo: {workdir}")
    
    # Guardar configuración
    config_path = workdir / "config.py"
    mmdet_config.dump(str(config_path))
    print(f"💾 Configuración guardada en: {config_path}")
    
else:
    print("❌ No se puede continuar sin datos válidos")


In [ ]:
# Iniciar entrenamiento V1 con backup automático
if train_json and val_json and test_json:
    
    print("🚀 Iniciando entrenamiento Deformable DETR V1 (Optimizado)...")
    print("🔥 Características V1:")
    print(f"  - Batch size optimizado: {detr_config.batch_size}")
    print(f"  - Val batch size optimizado: {detr_config.val_batch_size}")
    print(f"  - Workers optimizados: {detr_config.workers}")
    print(f"  - Guardado cada {detr_config.save_period} épocas")
    print(f"  - Backup en Drive cada {detr_config.drive_backup_period} épocas")
    print(f"  - Recuperación automática: {detr_config.resume_training}")
    print(f"⏱️  Tiempo estimado: {detr_config.epochs * 5} minutos (aproximado)")
    
    try:
        # Verificar si hay checkpoint para reanudar
        resume_path = None
        if detr_config.resume_training:
            # Buscar en Colab primero
            work_dir = Path(detr_config.work_dir)
            if work_dir.exists():
                checkpoints = list(work_dir.glob("*.pth"))
                if checkpoints:
                    resume_path = str(max(checkpoints, key=lambda x: x.stat().st_mtime))
                    print(f"🔄 Reanudando desde checkpoint local: {resume_path}")
            else:
                # Buscar en Drive
                resume_path = resume_training_from_drive()
                if resume_path:
                    print(f"🔄 Reanudando desde checkpoint en Drive: {resume_path}")
        
        # Configurar reanudación si es necesario
        if resume_path:
            mmdet_config.resume = True
            mmdet_config.load_from = resume_path
        
        # Crear runner
        runner = Runner.from_cfg(mmdet_config)
        
        # Iniciar backup automático en segundo plano
        print("🔄 Iniciando backup automático en segundo plano...")
        start_auto_backup()
        
        # Iniciar entrenamiento
        print("🚀 Iniciando entrenamiento...")
        runner.train()
        
        # Detener backup automático
        stop_auto_backup()
        
        print("✅ Entrenamiento completado exitosamente!")
        print(f"📁 Resultados guardados en: {detr_config.work_dir}")
        
        # Hacer backup final
        print("🔄 Realizando backup final...")
        backup_to_drive(epoch="final")
        
    except Exception as e:
        # Detener backup automático en caso de error
        stop_auto_backup()
        print(f"❌ Error durante el entrenamiento: {e}")
        print("💡 Verifica que los datos estén correctamente configurados")
        
else:
    print("❌ No se puede entrenar sin datos válidos")


## 🚀 Instrucciones de Uso - V1 Optimizado

### 📋 **Antes de Ejecutar:**
1. **Montar Google Drive** (se hace automáticamente)
2. **Verificar que tienes Colab Pro/Pro+** para mejor rendimiento
3. **Configurar datos** en la estructura correcta

### 🔥 **Características V1:**
- **Backup automático**: Los checkpoints se guardan en Drive cada 5 épocas
- **Recuperación automática**: Si se interrumpe, reanuda desde el último checkpoint
- **Configuración optimizada**: Batch size 8, workers 8 para mayor velocidad
- **Monitoreo en tiempo real**: Usa `show_training_status()` para ver progreso

### 📊 **Comandos Útiles Durante el Entrenamiento:**
```python
# Ver estado del entrenamiento
show_training_status()

# Hacer backup manual
backup_to_drive()

# Verificar backups en Drive
monitor_training_progress()
```

### ⚠️ **Importante:**
- **NO cierres la pestaña** de Colab durante el entrenamiento
- **Los backups se hacen automáticamente** cada 5 épocas
- **Si se interrumpe**, simplemente ejecuta de nuevo y reanudará automáticamente
- **Los checkpoints se guardan en**: `/content/drive/MyDrive/aerial-wildlife-count-results/deformable_detr_v1/`

### 🎯 **Optimizaciones de Velocidad:**
- **Batch size**: 8 (vs 4 original)
- **Val batch size**: 4 (vs 2 original)
- **Workers**: 8 (vs 4 original)
- **Save period**: 1 época (vs 10 original)
- **Early stopping**: 5 épocas de paciencia


## 📊 Evaluación y Visualización

## 📊 Cálculo de Métricas de Clasificación


In [ ]:
def calculate_mmdet_classification_metrics(log_file_path, classes):
    """
    Calcular métricas de clasificación para MMDetection
    
    Args:
        log_file_path: Ruta al archivo .log.json de MMDetection
        classes: Lista de nombres de clases
    
    Returns:
        dict: Métricas por clase y generales
    """
    import json
    from pathlib import Path
    
    log_file = Path(log_file_path)
    
    if not log_file.exists():
        print(f"❌ No se encontró archivo de log: {log_file}")
        return None
    
    # Leer todas las líneas del log
    with open(log_file, 'r') as f:
        lines = f.readlines()
    
    # Buscar la última evaluación (modo 'val')
    val_metrics = None
    for line in reversed(lines):
        try:
            data = json.loads(line.strip())
            if data.get('mode') == 'val':
                val_metrics = data
                break
        except:
            continue
    
    if not val_metrics:
        print("❌ No se encontraron métricas de validación en el log")
        return None
    
    metrics = {
        'per_class': {},
        'overall': {}
    }
    
    # Métricas generales
    metrics['overall']['bbox_mAP'] = val_metrics.get('coco/bbox_mAP', 'N/A')
    metrics['overall']['bbox_mAP_50'] = val_metrics.get('coco/bbox_mAP_50', 'N/A')
    metrics['overall']['bbox_mAP_75'] = val_metrics.get('coco/bbox_mAP_75', 'N/A')
    
    # Métricas por clase (si están disponibles)
    for i, class_name in enumerate(classes):
        class_key = f'coco/bbox_mAP_{class_name}'
        if class_key in val_metrics:
            metrics['per_class'][class_name] = {
                'mAP': val_metrics[class_key]
            }
    
    return metrics

def print_mmdet_classification_metrics(metrics):
    """Imprimir métricas de clasificación de MMDetection"""
    print("=" * 80)
    print("MÉTRICAS DE CLASIFICACIÓN - MMDetection")
    print("=" * 80)
    
    if 'per_class' in metrics and metrics['per_class']:
        print("\n📊 MÉTRICAS POR CLASE:")
        print("-" * 80)
        for class_name, class_metrics in metrics['per_class'].items():
            print(f"\n{class_name}:")
            for metric_name, value in class_metrics.items():
                print(f"  {metric_name}: {value:.4f}" if isinstance(value, float) else f"  {metric_name}: {value}")
    
    print("\n" + "=" * 80)
    print("📊 MÉTRICAS GENERALES:")
    print("-" * 80)
    for metric_name, value in metrics['overall'].items():
        print(f"{metric_name.upper():20s}: {value:.4f}" if isinstance(value, float) else f"{metric_name.upper():20s}: {value}")
    print("=" * 80)

def save_mmdet_classification_metrics(metrics, save_dir):
    """Guardar métricas de MMDetection"""
    import json
    import pandas as pd
    from pathlib import Path
    
    save_dir = Path(save_dir)
    
    # Guardar como JSON
    json_path = save_dir / "classification_metrics.json"
    with open(json_path, 'w') as f:
        json.dump(metrics, f, indent=2)
    print(f"✅ Métricas guardadas en JSON: {json_path}")
    
    # Guardar métricas generales como CSV
    overall_df = pd.DataFrame([metrics['overall']])
    csv_overall_path = save_dir / "classification_metrics_overall.csv"
    overall_df.to_csv(csv_overall_path, index=False)
    print(f"✅ Métricas generales guardadas en CSV: {csv_overall_path}")
    
    if metrics.get('per_class'):
        per_class_df = pd.DataFrame(metrics['per_class']).T
        csv_path = save_dir / "classification_metrics_per_class.csv"
        per_class_df.to_csv(csv_path)
        print(f"✅ Métricas por clase guardadas en CSV: {csv_path}")

print("✅ Funciones de métricas de clasificación MMDetection cargadas")


In [ ]:
# Calcular y mostrar métricas
work_dir = Path(detr_config.work_dir)
log_files = list(work_dir.glob("*.log.json"))

if log_files:
    latest_log = max(log_files, key=lambda x: x.stat().st_mtime)
    print(f"🔍 Calculando métricas desde: {latest_log.name}")
    metrics = calculate_mmdet_classification_metrics(latest_log, detr_config.classes)
    if metrics:
        print_mmdet_classification_metrics(metrics)
        save_mmdet_classification_metrics(metrics, work_dir)
else:
    print("❌ No se encontraron archivos de log")
    print("💡 Ejecuta el entrenamiento primero para generar métricas")


In [ ]:
# Función para evaluar el modelo entrenado
def evaluate_model():
    """Evaluar el modelo en el conjunto de test"""
    
    # Buscar el mejor checkpoint
    checkpoint_dir = Path(detr_config.work_dir)
    checkpoints = list(checkpoint_dir.glob("*.pth"))
    
    if not checkpoints:
        print("❌ No se encontraron checkpoints")
        return None
    
    # Usar el último checkpoint o el mejor
    best_checkpoint = None
    for ckpt in checkpoints:
        if "best" in ckpt.name:
            best_checkpoint = ckpt
            break
    
    if not best_checkpoint:
        best_checkpoint = sorted(checkpoints)[-1]  # Último checkpoint
    
    print(f"📊 Evaluando con checkpoint: {best_checkpoint}")
    
    # Cargar modelo
    model = init_detector(str(config_path), str(best_checkpoint), device=detr_config.device)
    
    # Evaluar en test
    test_results = model.test(
        data=mmdet_config.test_dataloader.dataset,
        metric="bbox"
    )
    
    print("✅ Evaluación completada")
    return test_results, best_checkpoint

# Ejecutar evaluación si el entrenamiento fue exitoso
if train_json and val_json and test_json:
    try:
        test_results, best_ckpt = evaluate_model()
        print(f"🎯 Mejor checkpoint: {best_ckpt}")
    except Exception as e:
        print(f"❌ Error en evaluación: {e}")

In [ ]:
# Función para visualizar predicciones
def visualize_predictions(model, image_path, save_path=None):
    """Visualizar predicciones en una imagen"""
    
    # Realizar inferencia
    result = inference_detector(model, image_path)
    
    # Visualizar
    from mmdet.visualization import DetLocalVisualizer
    visualizer = DetLocalVisualizer()
    
    # Cargar imagen
    img = cv2.imread(str(image_path))
    
    # Mostrar predicciones
    visualizer.add_datasample(
        "result",
        img,
        data_sample=result,
        draw_gt=False,
        wait_time=0,
        out_file=save_path
    )
    
    return result

# Función para probar el modelo en imágenes de ejemplo
def test_model_on_samples():
    """Probar el modelo en algunas imágenes de ejemplo"""
    
    if not (train_json and val_json and test_json):
        print("❌ No hay modelo entrenado para probar")
        return
    
    # Usar el directorio de imágenes de test ya determinado
    if not test_img_dir.exists():
        print("❌ No se encontró directorio de imágenes de test")
        return
    
    # Tomar algunas imágenes de ejemplo
    image_files = list(test_img_dir.glob("*.jpg"))[:3]  # Primeras 3 imágenes
    
    if not image_files:
        print("❌ No se encontraron imágenes de test")
        return
    
    print(f"🖼️  Probando modelo en {len(image_files)} imágenes...")
    
    # Cargar modelo
    try:
        model = init_detector(str(config_path), str(best_ckpt), device=detr_config.device)
        
        for i, img_path in enumerate(image_files):
            print(f"  Procesando imagen {i+1}: {img_path.name}")
            
            # Visualizar predicciones
            result = visualize_predictions(model, img_path)
            
            # Mostrar estadísticas
            if hasattr(result, "pred_instances"):
                num_detections = len(result.pred_instances)
                print(f"    Detecciones: {num_detections}")
        
        print("✅ Pruebas completadas")
        
    except Exception as e:
        print(f"❌ Error probando modelo: {e}")

# Ejecutar pruebas si hay modelo entrenado
test_model_on_samples()

## 💾 Guardar y Descargar Resultados

In [ ]:
# Función para guardar resultados en Google Drive
def save_to_drive():
    """Guardar resultados del entrenamiento en Google Drive"""
    
    if not os.path.exists("/content/drive"):
        print("❌ Google Drive no está montado")
        return False
    
    # Crear directorio en Drive
    drive_results_dir = "/content/drive/MyDrive/aerial-wildlife-count-results"
    os.makedirs(drive_results_dir, exist_ok=True)
    
    # Copiar directorio de trabajo
    import shutil
    try:
        shutil.copytree(detr_config.work_dir, f"{drive_results_dir}/deformable_detr_{detr_config.backbone}", dirs_exist_ok=True)
        print(f"✅ Resultados guardados en: {drive_results_dir}/deformable_detr_{detr_config.backbone}")
        return True
    except Exception as e:
        print(f"❌ Error guardando en Drive: {e}")
        return False

# Función para descargar archivos
def download_results():
    """Descargar archivos importantes del entrenamiento"""
    
    workdir = Path(detr_config.work_dir)
    
    # Archivos importantes a descargar
    important_files = [
        "config.py",
        "*.log",
        "*.pth"
    ]
    
    print("📥 Descargando archivos importantes...")
    
    for pattern in important_files:
        files_to_download = list(workdir.glob(pattern))
        for file_path in files_to_download:
            if file_path.is_file():
                print(f"  Descargando: {file_path.name}")
                files.download(str(file_path))

# Opciones para guardar resultados
print("💾 Opciones para guardar resultados:")
print("1. Guardar en Google Drive")
print("2. Descargar archivos")
print("3. Ambas opciones")

choice = input("Selecciona opción (1, 2, o 3): ").strip()

if choice in ["1", "3"]:
    save_to_drive()

if choice in ["2", "3"]:
    download_results()

if choice not in ["1", "2", "3"]:
    print("❌ Opción inválida")

## 🎉 ¡Entrenamiento Completado!

### 📋 Resumen del Entrenamiento
- **Modelo**: Deformable DETR con backbone {detr_config.backbone}
- **Tamaño de imagen**: {detr_config.imgsz}x{detr_config.imgsz}
- **Épocas**: {detr_config.epochs}
- **Early Stopping**: {detr_config.patience} épocas de paciencia
- **Clases detectadas**: {len(detr_config.classes)} especies de animales

### 📊 Próximos Pasos
1. **Evaluar métricas**: Revisar mAP, precision, recall
2. **Ajustar hiperparámetros**: Si es necesario mejorar el rendimiento
3. **Exportar modelo**: Convertir a ONNX o TorchScript para deployment
4. **Probar en nuevas imágenes**: Validar en datos no vistos

### 🔧 Configuración Personalizada
Para modificar parámetros, edita la clase `DeformableDETRConfig` en la celda de configuración:
- Cambiar backbone: `"swin_t"` o `"resnet50"`
- Ajustar épocas: `epochs = 100`
- Modificar tamaño de imagen: `imgsz = 1024`
- Cambiar batch size: `batch_size = 8`
- Ajustar early stopping: `patience = 10` (más paciencia) o `patience = 3` (menos paciencia)

### 📚 Recursos Adicionales
- [Documentación MMDetection](https://mmdetection.readthedocs.io/)
- [Deformable DETR Paper](https://arxiv.org/abs/2010.04159)
- [DETR Paper](https://arxiv.org/abs/2005.12872)
